In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
# Chemin vers le fichier CSV
filename = "newGPL18545_humanexome-12v1-1_a.csv"

# Charger le fichier CSV dans un dataframe pandas
df = pd.read_csv(filename)

# Supprimer les 23 dernières lignes
df = df.iloc[:-1]

# Écrire les données modifiées dans le fichier CSV
df.to_csv("newGPL18545_humanexome-12v1-1_a.csv", index=False)

In [13]:
#Une fonction qui supprime les crochéts
def remove_ends(input_string):
    if len(input_string) < 3:
        return ""
    else:
        return input_string[1:-1]


In [31]:
def remove_ends_column(df, column_name):
    # Créer une nouvelle colonne avec les valeurs modifiées
    new_column = []
    for i in df.index:
        value = df.loc[i, column_name]
        if isinstance(value, str):
            new_value = remove_ends(value)
        else:
            new_value = ""
        new_column.append(new_value)
    # Remplacer la colonne existante par la nouvelle colonne
    df[column_name] = new_column
    return df


remove_ends_column(genotypes, "SNP").head()

In [48]:
#Une fonction permettant d'extraire les SNPs des individus connaissant leurs identifiant de référence
def extract_row_by_id(df, id, id_column):
    row = df.loc[df[id_column] == id]
    return row

In [98]:
def extract_cell_value(row, col_index):
    # Récupération de la valeur de la cellule
    cell_value = row.iloc[0,col_index]
    return cell_value

In [73]:
### Chemin vers le fichier CSV contenant les données de génotypage Illumina
GPL_file = "newGPL18545_humanexome-12v1-1_a.csv"
GSE_file= "GSE148375_processed_data.txt"

# Lecture des données de génotypage à partir du fichier CSV
GPL = pd.read_csv(GPL_file, sep=",")
GSE = pd.read_csv(GSE_file, sep="\t")

/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
from tqdm import tqdm

In [80]:
remove_ends_column(GPL, "SNP")

# Conversion des génotypes au format VCF
hapmap = pd.DataFrame(columns=["exm#", "alleles", "chrom", "pos", "strand", "assembly#", "center",
                               "protLSID", "assayLSID", "panelLSID", "QCcode"])

# supprimer la première colonne de GPL
GSE_prime=GSE.copy()
GSE_prime.drop(GSE_prime.columns[0], axis=1, inplace=True)
# insérer les patients à la fin de hapmap
hapmap = pd.concat([hapmap, GSE_prime], ignore_index=True)

In [81]:
hapmap.head()

,exm#,alleles,chrom,pos,strand,assembly#,center,protLSID,assayLSID,panelLSID,...,215606,215753,215762,215763,215764,215769,215771,215772,215773,215827
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,NC,GG,GG,GG,GG,GG,GG,GG,GG
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,AA,GG,GG,GG,GG,GG,GG,GG,GG
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,NC,GG,GG,GG,GG,GG,GG,GG,GG
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,AG,GG,GG,GG,GG,GG,GG,GG,GG
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,NC,GG,GG,GG,GG,GG,GG,GG,GG


In [89]:
GPL_prime=GPL.copy()
row=extract_row_by_id(GPL_prime, "exm2268640-0_B_F_1984844585","IlmnID")

In [ ]:
#hapmap = ajouter_colonnes(hapmap)

for i in tqdm(range(0, len(GPL.index))):
    # Conversion des génotypes en format VCF
    hapmap.loc[i,"exm#"] = GPL.loc[i,"IlmnID"]
    hapmap.loc[i, "alleles"] = GPL.loc[i,"SNP"]
    hapmap.loc[i,"chrom"] = GPL.loc[i,"Chr"]
    hapmap.loc[i,"pos"] = GPL.loc[i,"MapInfo"]
    hapmap.loc[i,"strand"] = GPL.loc[i,"RefStrand"]
    hapmap.loc[i,"assembly#"] = "NA"
    hapmap.loc[i,"center"] = "NA"
    hapmap.loc[i,"protLSID"] = "NA"
    hapmap.loc[i,"assayLSID"] = "NA"
    hapmap.loc[i,"panelLSID"] = "NA"
    hapmap.loc[i,"QCcode"] = "NA"
    for j in range (11,len(hapmap.columns)):
        line=extract_row_by_id(GSE, GPL.loc[i,"IlmnID"], "ID_REF")
        hapmap.iloc[i,j] = extract_cell_value(line,j-11)

# Écriture des génotypes convertis en format hapmap dans un fichier
hapmap.to_csv("data.hapmap", sep="\t", index=False)




  0%|          | 0/242901 [00:00<?, ?it/s]


  0%|          | 1/242901 [00:32<2218:04:05, 32.87s/it]


  0%|          | 2/242901 [01:04<2197:09:56, 32.56s/it]


  0%|          | 3/242901 [01:35<2170:01:13, 32.16s/it]


  0%|          | 4/242901 [02:07<2157:06:33, 31.97s/it]


  0%|          | 5/242901 [02:42<2223:11:56, 32.95s/it]


  0%|          | 6/242901 [03:17<2253:16:27, 33.40s/it]


  0%|          | 7/242901 [03:48<2212:17:29, 32.79s/it]


  0%|          | 8/242901 [04:19<2179:01:03, 32.30s/it]


  0%|          | 9/242901 [04:52<2196:54:50, 32.56s/it]


  0%|          | 10/242901 [05:25<2196:22:56, 32.55s/it]


  0%|          | 11/242901 [05:57<2190:19:18, 32.46s/it]


  0%|          | 12/242901 [06:29<2169:16:30, 32.15s/it]


  0%|          | 13/242901 [06:59<2141:43:38, 31.74s/it]


  0%|          | 14/242901 [07:30<2122:06:15, 31.45s/it]


  0%|          | 15/242901 [08:02<2136:12:17, 31.66s/it]


  0%|          | 16/242901 [08:37<2203:30:30, 32.66s/it]


  0%|          | 

  0%|          | 120/242901 [1:08:40<2110:06:39, 31.29s/it]


  0%|          | 121/242901 [1:09:12<2111:05:07, 31.30s/it]


  0%|          | 122/242901 [1:09:47<2183:57:42, 32.38s/it]


  0%|          | 123/242901 [1:10:23<2256:16:10, 33.46s/it]


  0%|          | 124/242901 [1:10:54<2223:03:32, 32.96s/it]


  0%|          | 125/242901 [1:11:29<2253:31:47, 33.42s/it]


  0%|          | 126/242901 [1:12:04<2293:26:50, 34.01s/it]


  0%|          | 127/242901 [1:12:36<2250:48:42, 33.38s/it]


  0%|          | 128/242901 [1:13:08<2210:57:03, 32.79s/it]


  0%|          | 129/242901 [1:13:39<2187:49:11, 32.44s/it]


  0%|          | 130/242901 [1:14:11<2181:22:50, 32.35s/it]


  0%|          | 131/242901 [1:14:45<2207:47:23, 32.74s/it]


  0%|          | 132/242901 [1:15:20<2255:42:52, 33.45s/it]


  0%|          | 133/242901 [1:15:54<2262:31:09, 33.55s/it]


  0%|          | 134/242901 [1:16:29<2283:29:46, 33.86s/it]